In [1]:
import pickle
import os
import pandas as pd
import numpy as np
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBRegressor

In [2]:
hd_train = pd.read_csv("heart_2020_cleaned.csv")
cardio_train = pd.read_csv("cardio_train.csv", sep=';')

In [3]:
hd_train

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [4]:
cardio_train

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [6]:
cardio_train['age']=(cardio_train['age']/365).round(0)
cardio_train['bmi']=(cardio_train['weight']/(cardio_train['height']/100*cardio_train['height']/100)).round(0)
cardio_train = cardio_train.drop(columns = ['id'])
cardio_train['gender'] = cardio_train['gender'].map({1:0,2:1})
cardio_train.drop(index=0)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
1,55.0,0,156,85.0,140,90,3,1,0,0,1,1,35.0
2,52.0,0,165,64.0,130,70,3,1,0,0,0,1,24.0
3,48.0,1,169,82.0,150,100,1,1,0,0,1,1,29.0
4,48.0,0,156,56.0,100,60,1,1,0,0,0,0,23.0
5,60.0,0,151,67.0,120,80,2,2,0,0,0,0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,53.0,1,168,76.0,120,80,1,1,1,0,1,0,27.0
69996,62.0,0,158,126.0,140,90,2,2,0,0,1,1,50.0
69997,52.0,1,183,105.0,180,90,3,1,0,1,0,1,31.0
69998,61.0,0,163,72.0,135,80,1,2,0,0,0,1,27.0


In [9]:
hd_train = hd_train.drop(columns = ['Race'])
hd_train = hd_train.drop(index=0)

In [11]:
hd_train.replace(['18-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80 or older'],
                 ['18','25','30','35','40','45','50','55','60','65','70','75','80'],inplace = True)
hd_train['Diabetic'] = hd_train['HeartDisease'].map({'No, borderline diabetes':'No','Yes (during pregnancy)':'Yes','No':'No','Yes':'Yes'})
hd_train['HeartDisease'] = hd_train['HeartDisease'].map({'Yes':1,'No':0})
hd_train['Smoking'] = hd_train['Smoking'].map({'Yes':1,'No':0})
hd_train['AlcoholDrinking'] = hd_train['AlcoholDrinking'].map({'Yes':1,'No':0})
hd_train['Stroke'] = hd_train['Stroke'].map({'Yes':1,'No':0})
hd_train['DiffWalking'] = hd_train['DiffWalking'].map({'Yes':1,'No':0})
hd_train['Diabetic'] = hd_train['Diabetic'].map({'Yes':1,'No':0})
hd_train['PhysicalActivity'] = hd_train['PhysicalActivity'].map({'Yes':1,'No':0})
hd_train['Asthma'] = hd_train['Asthma'].map({'Yes':1,'No':0})
hd_train['KidneyDisease'] = hd_train['KidneyDisease'].map({'Yes':1,'No':0})
hd_train['SkinCancer'] = hd_train['SkinCancer'].map({'Yes':1,'No':0})
hd_train['Sex'] = hd_train['Sex'].map({'Male':1,'Female':0})
hd_train['GenHealth'] = hd_train['GenHealth'].map({'Poor':1,'Fair':2,'Good':3,'Very good':4,'Excellent':5})

In [12]:
num = int(input("1-Heart Disease, 2-Cardiovascular Disease:"))

1-Heart Disease, 2-Cardiovascular Disease:2


In [13]:
if(num==1):
    BMI = float(input("Enter BMI:\n"))
    Smoking = int(input("Have you smoked at least 100 cigarettes in your entire life?:\n(1-True,0-False)\n")) 
    AlcoholDrinking = int(input("Heavy drinkers?:\n(1-True,0-False)\n"))
    Stroke = int(input("Ever had a stroke?:\n(1-True,0-False)\n"))
    PhysicalHealth =  int(input("How many days during the past 30 days was your physical health not good?:\n"))
    MentalHealth = int(input("How many days during the past 30 days was your mental health not good?:\n"))
    DiffWalking = int(input("Do you have serious difficulty walking or climbing stairs?:\n(1-True,0-False)\n"))
    Sex = int(input("Gender:\n(Male-1,Female-0)\n"))
    AgeCategory = int(input("Age:\n"))
    Diabetic = int(input("Ever had diabetes?:\n(1-True,0-False)\n"))
    PhysicalActivity = int(input("Done physical activity or exercise during the past 30 days?:\n(1-True,0-False)\n"))
    GenHealth = int(input("Would you say your health in general is:\n(1-Poor,2-Fair,3-Good,4-Very Good,5-Excellent)\n"))
    SleepTime = int(input("How many hours of sleep do you get in a 24-hour period?:\n"))
    Asthma = int(input("Ever had asthma?:\n(1-True,0-False)\n"))
    KidneyDisease = int(input("Ever had kidney disease?:\n(1-True,0-False)\n"))
    SkinCancer = int(input("Ever had skin cancer?:\n(1-True,0-False)\n"))

    User_data = [[BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,
            Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer]]
    print(User_data)
    
elif(num == 2):
    age = int(input("Age:\n"))
    gender = int(input("Gender:\n(1-Male,0-Female)\n"))
    height = float(input("Height:(in m)\n"))
    weight = float(input("Weight:(in kg)\n"))
    ap_hi = int(input("Systolic Blood Pressure:\n"))
    ap_lo = int(input("Diastolic Blood Pressure:\n"))
    cholesterol = int(input("Cholesterol Level:\n(1-Normal,2-Above Normal,3-Well Above Normal)"))
    gluc = int(input("Glucose Level:\n(1-Normal,2-Above Normal,3-Well Above Normal)"))
    smoke = int(input("Are you smoking now?:\n(1-True,0-False)"))
    alco = int(input("Are you drinking alcohol?:\n(1-True,0-False)"))
    active = int(input("Are you doing physical activity?:\n(1-True,0-False)"))
    
    bmi = weight/(height*height)
    
    User_data = [[age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,bmi]]

Age:
50
Gender:
(1-Male,0-Female)
1
Height:(in m)
1.68
Weight:(in kg)
62
Systolic Blood Pressure:
110
Diastolic Blood Pressure:
80
Cholesterol Level:
(1-Normal,2-Above Normal,3-Well Above Normal)1
Glucose Level:
(1-Normal,2-Above Normal,3-Well Above Normal)1
Are you smoking now?:
(1-True,0-False)0
Are you drinking alcohol?:
(1-True,0-False)0
Are you doing physical activity?:
(1-True,0-False)1


In [14]:
def Output(rforest,X,y): 
    # Training the model on 100% Data available
    Final_XGB_Model=rforest.fit(X,y)

    # Saving the Python objects as serialized files can be done using pickle library
    # Here let us save the Final model
    with open('./Final_XGB_Model.pkl', 'wb') as fileWriteStream:
        pickle.dump(Final_XGB_Model, fileWriteStream)
        # Don't forget to close the filestream!
        fileWriteStream.close()

    #print('pickle file of Predictive Model is saved at Location:',os.getcwd())

In [15]:
def FunctionPredictResult(InputData,num):
    ### Sandardization of data ###
    Num_Inputs=InputData.shape[0]
    
    # Making sure the input data has same columns as it was used for training the model
    # Also, if standardization/normalization was done, then same must be done for new input
    
    # Appending the new data with the Training data
    DataForML=pd.read_pickle('Final_XGB_Model.pkl')
    InputData=InputData.append([DataForML])
    
    # Generating dummy variables for rest of the nominal variables
    #InputData=pd.get_dummies(InputData)
            
    # Maintaining the same order of columns as it was during the model training
    if(num == 2):
        Predictors=[ 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
           'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'bmi']
    elif(num == 1):
        Predictors=[  'BMI', 'Smoking', 'AlcoholDrinking', 'Stroke',
           'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory',
           'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime', 'Asthma',
           'KidneyDisease', 'SkinCancer']
    
    # Generating the input values to the model
    X=InputData[Predictors].values[0:Num_Inputs]

    #PredictorScaler=MinMaxScaler()
    # Generating the standardized values of X since it was done while model training also
    #PredictorScalerFit=PredictorScaler.fit(X)
    #X=PredictorScalerFit.transform(X)
    # Loading the Function from pickle file
    with open('Final_XGB_Model.pkl', 'rb') as fileReadStream:
        PredictionModel=pickle.load(fileReadStream)
        # Don't forget to close the filestream!
        fileReadStream.close()
            
    # GenPrice Predictions
    Prediction=PredictionModel.predict(X)
   
    PredictionResult=pd.DataFrame(Prediction, columns=['Prediction'])
    return(round(PredictionResult))

In [16]:
def predict(rforest,X,y,num):
    Output(rforest,X,y)
    # Calling the function for some new data
    NewSampleData=pd.DataFrame(
    
    data=User_data,
    columns=['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
       'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'bmi'
       ])

    NewSampleData
    #value = FunctionPredictResult(InputData= NewSampleData)
    # Calling the Function for prediction
    print(FunctionPredictResult(NewSampleData,num))

def predict1(rforest1,X1,y1,num):
    Output(rforest1,X1,y1)
    # Calling the function for some new data
    NewSampleData1=pd.DataFrame(
    data=User_data,
    columns=['BMI', 'Smoking', 'AlcoholDrinking', 'Stroke',
       'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory',
       'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime', 'Asthma',
       'KidneyDisease', 'SkinCancer'])

    NewSampleData1
    #value = FunctionPredictResult(InputData= NewSampleData)
    # Calling the Function for prediction
    print(FunctionPredictResult(NewSampleData1,num))

In [17]:
if(num == 2):
    TargetVariable='cardio'
    Predictors=['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
           'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'bmi'
           ]
    data_Numeric=cardio_train

    X=data_Numeric[Predictors].values
    y=data_Numeric[TargetVariable].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    
elif(num == 1):
    TargetVariable1='HeartDisease'
    Predictors1=['BMI', 'Smoking', 'AlcoholDrinking', 'Stroke',
           'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory',
           'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime', 'Asthma',
           'KidneyDisease', 'SkinCancer']
    data_Numeric1=hd_train
    X1=data_Numeric1[Predictors1].values
    y1=data_Numeric1[TargetVariable1].values
    
    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.1)

In [18]:
if (num == 2):
    # Create the Random Forest object
    rforest = RandomForestClassifier()
    # Fit Random Forest on Train Data
    rforest.fit(X_train, y_train)
    # Predict the Response corresponding to Predictors
    y_train_pred = rforest.predict(X_train)
    
    print("Train Data")
    print("Accuracy  :\t", rforest.score(X_train, y_train))
    print("Test Data")
    print("Accuracy  :\t", rforest.score(X_test, y_test))
    
elif(num == 1):
    # Create the Random Forest object
    rforest1 = RandomForestClassifier()
    # Fit Random Forest on Train Data
    rforest1.fit(X1_train, y1_train)
    # Predict the Response corresponding to Predictors
    y1_train_pred = rforest1.predict(X1_train)
    
    print("Train Data")
    print("Accuracy  :\t", rforest1.score(X1_train, y1_train))
    print("Test Data")
    print("Accuracy  :\t", rforest1.score(X1_test, y1_test))
    

Train Data
Accuracy  :	 0.9744761904761905
Test Data
Accuracy  :	 0.714



In [19]:
if(num==2):
   predict(rforest,X,y,num)
elif(num==1):
   predict1(rforest1,X1,y1,num)

   Prediction
0           0


/var/folders/pr/1qtjlkcn2g36y0t6_p6wd3f80000gn/T/ipykernel_2709/1639569504.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  InputData=InputData.append([DataForML])
